In [8]:
import requests
import os
import argparse
import yaml
import re
import subprocess
import json
import urllib

In [55]:
"""
This function is useful in parsing the yaml file for an entire canvas course resulting in the extraction of a dictionary with the module id numbers as the keys and the lesson detail as the values.
"""
def lessons_list(yaml_file):
    with open(yaml_file) as f:
        data = yaml.load(f, Loader=yaml.FullLoader)
    lesson_data = {}

    for i in data[':modules']:
        lesson_data[i[':id']] = []
        for j in i[':lessons']:
            if j['type'] == 'Assignment' or j['type'] == 'Page':
                lesson_data[i[':id']].append(j['url'])
            else:
                continue
            
    return lesson_data

In [42]:
def equation(result):
    """
    Takes in the result of a regex match and returns the canvas string for rendering latex
    """
    string = result.group()
    item = string.split('?math=')[1].strip('" />')
    item_encoded = "https://learning.flatironschool.com/equation_images/"+urllib.parse.quote(item)
    return f"""<img class="equation_image" title="{item}" src="{item_encoded}" alt="{"LaTeX: "+item}" data-equation-content="{item}" />"""

In [49]:
def fix_canvas_latex(page_string):
    with open('test_file.txt', "r")as f:
        data = f.read()re.sub('<img src="https://render.githubusercontent.com/render/math\?math=.*?" />', equation, data)
    file = 
    with open('fixed_file.txt', 'w')as j:
        j.write(file)



In [153]:
"""
This function takes in a string and fixes the latex rendering for canvas 
Returns the fixed string as a string
"""
def fix_canvas(page_file):
    with open(page_file)as j:
        page_string = j.read()
    print(page_string[:100])
    contents = re.sub('<img src="https://render.githubusercontent.com/render/math\?math=.*?" />', equation, page_string)
    contents = re.sub('<img src="https://render.githubusercontent.com/render/math\?math=.*?"/>', equation, contents)
    contents = re.sub('<img src="https://render.githubusercontent.com/render/math\?math=.*?">', equation, contents)
    with open('fixed_file.txt', 'w')as f:
        f.write(contents)

In [129]:
"""
Takes a canvas lesson url and returns the lesson contents as a dictionary with the following keys:
name: the name of the lesson as it appears in the canvas page header

description: this is the html content of the lesson if it is a 'page'
body: this is the html content of the lesson if it is an 'assignment'

type: this is the type of lesson either a page or assignment

The function retrieves the lesson html content from canvas and returns that content as a string
"""
def canvas_details(canvas_url):    
    lesson_url = canvas_url.strip()
    command = ('github-to-canvas',  '--read-from-canvas', lesson_url)
    process = subprocess.Popen(command, stdout=subprocess.PIPE)
    raw_data = process.communicate()[0]
    data = yaml.safe_load(raw_data)
    lesson_type = data['type']
    if lesson_type == 'Page':
        if (len(re.findall('<img src="https://render.githubusercontent.com/render/math\?math=', data['body']))) == 0:
            print('Not a LaTeX Repo')
            lesson_type = 'Not LaTeX'
            file = None
            name = None
        else:
            file = data['body']
            name = data['title']
    elif lesson_type == 'Assignment':
        if (len(re.findall('<img src="https://render.githubusercontent.com/render/math\?math=', data['description']))) == 0:
            print('Not a LaTeX Repo')
            lesson_type = 'Not LaTeX'
            file = None
            name = None
        else:
            file = data['description']
            name = data['name']
    return lesson_url, file, lesson_type, name

In [142]:
"""
This takes the canvas_url, page_html, and lesson type and replaces the html in canvas with the new file 'page_html'
return is the response json object
"""
def update_details(canvas_url, file, lesson_type):
    with open(file, 'r')as f:
        page_html = f.read()
    if lesson_type == 'Not LaTeX':
        print("Not a LaTeX lesson")
    if lesson_type == 'Page':
        url = canvas_url
        payload = {'wiki_page[body]': page_html}
        head = {'Authorization': f"Bearer {os.environ.get('CANVAS_API_KEY')}"}
        response = requests.put(url=url, headers=head, data=payload)
        return response.json()

    if lesson_type == 'Assignment':
        url = canvas_url
        payload = {'assignment[description]': page_html}
        head = {'Authorization': f"Bearer {os.environ.get('CANVAS_API_KEY')}"}
        response = requests.put(url=url, headers=head, data=payload)
        return response.json()


In [169]:
def fix_course_latex(yaml_file):
    lessons = lessons_list(yaml_file=yaml_file)
    for j in lessons:
        for i in lessons[j]:
            lesson_url, data_temp, page_type, name = canvas_details(i)
            if page_type == 'Not LaTeX':
                print(f'{name} does not have any LaTeX formulas in it\nMoving on to the next item')
            else:
                with open('temp_file.txt', 'w')as j:
                    j.write(data_temp)
                print(f'Fixing {name}')
                fix_canvas('temp_file.txt')
                print(f'Updating {name} at \n {lesson_url}')
                update_details(canvas_url=lesson_url, file='fixed_file.txt', lesson_type=page_type)
                print(f'Sucess!! Done with {name}, moving on to the next one.')
        
        
        

In [170]:
fix_course_latex('full_course_download.yml')

Not a LaTeX Repo
None does not have any LaTeX formulas in it
Moving on to the next item
Not a LaTeX Repo
None does not have any LaTeX formulas in it
Moving on to the next item
Not a LaTeX Repo
None does not have any LaTeX formulas in it
Moving on to the next item
Not a LaTeX Repo
None does not have any LaTeX formulas in it
Moving on to the next item
Not a LaTeX Repo
None does not have any LaTeX formulas in it
Moving on to the next item
Not a LaTeX Repo
None does not have any LaTeX formulas in it
Moving on to the next item
Not a LaTeX Repo
None does not have any LaTeX formulas in it
Moving on to the next item
Not a LaTeX Repo
None does not have any LaTeX formulas in it
Moving on to the next item
Not a LaTeX Repo
None does not have any LaTeX formulas in it
Moving on to the next item
Not a LaTeX Repo
None does not have any LaTeX formulas in it
Moving on to the next item
Not a LaTeX Repo
None does not have any LaTeX formulas in it
Moving on to the next item
Not a LaTeX Repo
None does not h

In [168]:
lessons = lessons_list('full_course_download.yml')
lessons[28929]

['https://learning.flatironschool.com/api/v1/courses/4713/assignments/131387',
 'https://learning.flatironschool.com/api/v1/courses/4713/assignments/131384',
 'https://learning.flatironschool.com/api/v1/courses/4713/assignments/131383',
 'https://learning.flatironschool.com/api/v1/courses/4713/assignments/131360',
 'https://learning.flatironschool.com/api/v1/courses/4713/assignments/131406',
 'https://learning.flatironschool.com/api/v1/courses/4713/assignments/131404',
 'https://learning.flatironschool.com/api/v1/courses/4713/assignments/131358']

In [ ]:
for mod in mod_list:
    try:
        fix_mod(mod)
        mod_list.remove(mod)
        print(f'mod {mod} done')
    except:
        print(f'mod {mod} completed')